In [1]:
# Displaying graphs

import seaborn as sns
import plotly.io as pio
pio.renderers.default = 'iframe'


In [2]:
# Data

import pandas as pd
from datetime import datetime


# Charts

import plotly.express as px
import plotly.graph_objects as go



# Dash

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output


In [3]:
sales_df = pd.read_csv('supermarket_sales - Sheet1.csv')

In [4]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [5]:
sales_df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [6]:
print(sales_df[sales_df['Branch'] == 'A'].City.unique())
print(sales_df[sales_df['Branch'] == 'B'].City.unique())
print(sales_df[sales_df['Branch'] == 'C'].City.unique())

['Yangon']
['Mandalay']
['Naypyitaw']


In [7]:
sales_df['Product line'].unique()

array(['Health and beauty', 'Electronic accessories',
       'Home and lifestyle', 'Sports and travel', 'Food and beverages',
       'Fashion accessories'], dtype=object)

In [8]:
# Convert Date to datetime format

sales_df['Date'] = pd.to_datetime(sales_df['Date'])
sales_df['Time'] = pd.to_datetime(sales_df['Time'])


# Needed for two graphs
sales_df['day_name'] = sales_df['Date'].dt.day_of_week
sales_df['hour'] = sales_df['Time'].dt.hour
sales_df['month'] = sales_df['Date'].dt.month


# Create a dictionary to map month values to month names
month_names = {
    1: "January",
    2: "February",
    3: "March"
    # Add more months if needed
}

# Convert the Month column to month names
sales_df["month"] = sales_df["month"].map(month_names)


In [9]:
# The beginning of the previous month
previous_month = datetime.fromisoformat('2019-02-01')
# The beginning of the present month
present_month = datetime.fromisoformat('2019-03-01')

df_dict = {}



# Needed to simplify calculations and charting
sales_df['ones'] = 1

df_dict['prev_all'] = sales_df[(sales_df['Date'] >= previous_month) & (sales_df['Date'] < present_month)]
df_dict['pres_all'] = sales_df[sales_df['Date'] >= present_month]

city_list = ['Yangon', 'Mandalay', 'Naypyitaw']
section_list = ['cogs', 'Quantity', 'Rating', 'ones']



# Formation of a dictionary with dataframes
for i in list(df_dict):
    for z in city_list:
        i_df = df_dict[i]
        df_dict[i[:-3] + z] = i_df[i_df['City'] == z]

list(df_dict)


['prev_all',
 'pres_all',
 'prev_Yangon',
 'prev_Mandalay',
 'prev_Naypyitaw',
 'pres_Yangon',
 'pres_Mandalay',
 'pres_Naypyitaw']

In [10]:
#Creating a convenient dataframe with all the values you need for a dashboard

stat_dict = {'present':[False, True, False, False, False, True, True, True],
             'city':['all', 'all', 'Yangon', 'Mandalay', 'Naypyitaw', 'Yangon', 'Mandalay', 'Naypyitaw']}



for x in section_list:
    x_list = []
    for c in list(df_dict):
        c_df = df_dict[c][x]
        if x == 'Rating':
            ss = c_df.sum()/len(c_df)
        else:
            ss = c_df.sum()
            
        x_list.append(round(ss, 2))
    
    stat_dict[x] = x_list.copy()

stat_df = pd.DataFrame(stat_dict)
stat_df


,present,city,cogs,Quantity,Rating,ones
0,False,all,92589.88,1654,7.07,303
1,True,all,104243.34,1891,6.84,345
2,False,Yangon,28438.21,493,7.01,94
3,False,Mandalay,32785.02,624,7.01,109
4,False,Naypyitaw,31366.65,537,7.20,100
5,True,Yangon,35865.83,681,6.99,127
6,True,Mandalay,32949.85,596,6.65,112
7,True,Naypyitaw,35427.66,614,6.86,106


In [11]:
city_df_1 = stat_df[(stat_df['city'] != 'all') & (stat_df['present'] == True)]
city_df_2 = stat_df[(stat_df['city'] != 'all') & (stat_df['present'] == False)]

In [12]:
# It is necessary to calculate how many percent the indicator has increased or decreased
def num_str(n1, n2):
    if n1 > n2:
        perc = (n1-n2)/n2*100
        symb = '▲'
        col = 'green'
        
    elif n1 < n2:
        perc = (n2-n1)/n2*100
        symb = '▼'
        col = 'red'

    
    perc_symb = symb + ' ' + str(round(perc,2)) + '%'
    return [str(n1), str(n2), perc_symb, col]

In [13]:
selection_name = {'cogs':'Cost of goods sold', 
                  'Quantity':'Quantity of goods sold',
                  'Rating':'Rating of goods sold',
                  'ones':'Quantity of orders'}


emoji_s = {'cogs':'💲', 'Quantity':'📦', 'Rating':'⭐', 'ones':'🛍️'}



city_list = ['Yangon', 'Mandalay', 'Naypyitaw']
section_list = ['cogs', 'Quantity', 'Rating', 'ones']


for_present_green = {'color':'#009900',
                     'fontSize':'22px',
                     'fontFamily': 'Arial',
                     'textAlign':'start',
                     'margin':'0 auto',}


for_present_red = {'color':'#990000',
                   'fontSize':'22px',
                   'fontFamily': 'Arial',
                   'textAlign':'start',
                   'margin':'0 auto',}

for_previous_green = {'color':'#009900',
                      'fontSize':'20px',
                      'textAlign':'center',
                      'fontFamily': 'Arial',
                      'textAlign':'start',
                      'margin':'0 auto',}


for_previous_red = {'color':'#990000',
                    'fontSize':'20px',
                    'textAlign':'center',
                    'fontFamily': 'Arial',
                    'textAlign':'start',
                    'margin':'0 auto',}

for_percent_green = {'color':'#009900',
                     'fontSize':'24px',
                     'fontFamily': 'Arial',
                     'position': 'absolute',
                     'top':'70px',
                     'right':'10px',
                     'margin':'0 auto',}

for_percent_red = {'color':'#990000',
                   'fontSize':'24px',
                   'fontFamily': 'Arial',
                   'position': 'absolute',
                   'top':'70px',
                   'right':'10px',
                   'margin':'0 auto',}



pres_css_dict = {'green':for_present_green,
                 'red':for_present_red}

prev_css_dict = {'green':for_previous_green,
                 'red':for_previous_red}

percent_css_dict = {'green':for_percent_green,
                    'red':for_percent_red}



html_city_dict = {}


for_head_block = {'color':'#212121',
                  'fontSize':'28px',
                  'fontWeight':'bold',
                  'textAlign':'start',
                  'fontFamily': 'Arial',
                  'margin':'0 auto',}

for_descr_present = {'color':'#757575',
                      'fontSize':'20px',
                      'fontFamily': 'Arial',
                      'textAlign':'start',
                      'margin':'0 auto',}

for_stat_text = {'backgroundColor':'#FFFFFF',
                 'width':'320px',
                 'borderStyle': 'solid',
                 'borderWidth': '10px',
                 'borderColor': '#68228B',
                 'padding':'10px',
                 'position':'relative',
                 'margin':'15px auto 0',}

for_div_texts = {'display':'flex',
                 'flexDirection':'row',
                 'justifyContent':'space-evenly',}




# Forming html blocks with statistics
for v in city_list:
    row_df_1 = city_df_1[city_df_1['city'] == v]
    row_df_2 = city_df_2[city_df_2['city'] == v]
    
    section_block = []
    for b in section_list:
        row_1 = row_df_1[b].iloc[0]
        row_2 = row_df_2[b].iloc[0]
        res_mas = num_str(row_1, row_2)
        
        html_list = [html.P(children = selection_name[b], style = for_head_block),
                     html.P(children = 'In March', style = for_descr_present),
                     html.P(children = emoji_s[b] + ' ' + res_mas[0], style = pres_css_dict[res_mas[3]]),
                     html.P(children = 'In February', style = for_descr_present),
                     html.P(children = emoji_s[b] + ' ' + res_mas[1], style = prev_css_dict[res_mas[3]]),
                     html.P(children = res_mas[2], style = percent_css_dict[res_mas[3]])]
        
        section_block.append(html.Div(children = html_list.copy(), style = for_stat_text))
    
    html_city_dict[v] = html.Div(children = section_block, style = for_div_texts)
        

In [14]:

bar_2 = px.bar(data_frame=sales_df, 
             y='Product line',x='ones',color = 'City',orientation='h', barmode='group',
               labels={'ones':'Number of orders',
                       'Product line':'Product line'},
              title='Number of orders by Product line',
              template='simple_white')
bar_2.update_traces(opacity=0.6, marker_line_width=0)


In [15]:

bar_3 = px.bar(data_frame=sales_df, 
               x='day_name', y='ones',color = 'City', barmode='group',
               labels={'ones':'Number of orders',
                       'day_name':'Day of week'},
               title='Number of orders by day of week',
               template='simple_white')
bar_3.update_xaxes(
    ticktext=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
    tickvals=[0,1,2,3,4,5,6])
bar_3.update_traces(opacity=0.6, marker_line_width=0)


In [ ]:
app = dash.Dash(__name__)
server = app.server 
app.title = "Supermarket sales"


for_sale_smile = {'fontSize':'100px',
                  'margin':'30px auto 0',
                  'position': 'absolute',
                   'top':'4px',
                   'right':'250px',}

for_header_title = {'color':'#FFFFFF',
                    'fontSize':'80px',
                    'fontWeight':'bold',
                    'textAlign':'center',
                    'margin':'4 auto',}

for_header = {'backgroundColor': '#5D478B',
              'display':'flex',
              'flexDirection':'column',
              'justifyContent':'center',}


 
sales_text = html.Div(children=[html.P(children='🛒', className='game_smile', style=for_sale_smile ), 
                                html.H1(children='Supermarket Sales', 
                                        className='header_title', style=for_header_title),
                               ],
                      className='header', style=for_header)




for_main_texts = {'padding':'10px',
                  'width':'96%',
                  'border': '5px solid transparent',
                  'borderImage': 'linear-gradient(to right,#68228B 10%,  transparent 30%, transparent 70%, #68228B 90%)',
                  'borderImageSlice': '1',
                  'margin':'15px auto 0',}


for_radio_text = {'fontSize':'26px',
                  'padding':'2px',
                  'fontFamily': 'Arial',
                  'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
                  'marginLeft':'5px',
                  'marginRight':'5px',}


for_radio_div = {'display':'flex',
                 'flexDirection':'row',
                 'justifyContent':'center',}




text_div = html.Div(children=[dcc.RadioItems(id='radio-text', 
                                             options=[{'label':'Yangon',
                                                       'value':'Yangon'},
                                                      {'label':'Mandalay',
                                                       'value':'Mandalay'},
                                                      {'label':'Naypyitaw', 
                                                       'value':'Naypyitaw'}],
                                             value='Yangon',
                                             labelStyle=for_radio_text, style=for_radio_div),
                              html.Div(id='text-block-output')],
                    className='main_texts',
                    style=for_main_texts)



for_bar1_div = {'backgroundColor':'#FFFFFF',
                'width':'43%',
                'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
                'margin':'0px auto',}


for_bar1_drop = {'fontSize':'20px',
                 'fontFamily': 'Arial',}



bar1_div = html.Div(children=[dcc.Dropdown(id='dropdown1-bar',
                                           options=[{'label':'The number of orders (Gender)',
                                                     'value':'Gender'},
                                                    {'label':'The number of orders (City)',
                                                     'value':'City'},
                                                    {'label':'The number of orders (Customer type)',
                                                     'value':'Customer type'},
                                                    {'label':'The number of orders (Payment)',
                                                     'value':'Payment'}],
                                           value='Customer type',
                                           style=for_bar1_drop),
                              dcc.Graph(id='dropdown-bar1-output')],
                    className='bar1_div',
                    style=for_bar1_div)



for_box_drop = {'fontSize':'20px',
                'width':'200px',
                'fontFamily': 'Arial',}


for_box_radio = {'fontSize':'20px',
                 'marginLeft':'10px',
                 'fontFamily': 'Arial',}


for_box_radio_label = {'marginLeft':'5px',} 


for_knopki_div = {'display':'flex',
                  'flexDirection':'row',}



knopki_div = html.Div(children=[dcc.Dropdown(id='box-drop',
                                             options=[ 
                                                      {'label':'City',
                                                       'value':'City'},
                                                      {'label':'Customer type',
                                                       'value':'Customer type'},
                                                      {'label':'Product line',
                                                       'value':'Product line'},
                                                      {'label':'Payment',
                                                       'value':'Payment'}],
                                             value='Product line', 
                                             style=for_box_drop),
                                
                                dcc.RadioItems(id='box-radio',
                                               options=[{'label':'💲 Cost',
                                                         'value':'cogs'},
                                                        {'label':'📦 Goods',
                                                         'value':'Quantity'},
                                                        {'label':'⭐ Rating',
                                                         'value':'Rating'}],
                                               value='cogs', 
                                               labelStyle=for_box_radio_label,
                                               style=for_box_radio)],
                      style=for_knopki_div)

for_drop_div={"display": "flex", 
              "flex-direction": "row",
              'justifyContent':'space-evenly',
              'fontSize':'20px',
                 'fontFamily': 'Arial',
              'verticalAlign': 'middle',
              'marginTop': 15,
             }

dropdown_div = html.Div(children=[html.H3("City:",style={'marginTop': 9,}),dcc.Dropdown(id="city-dropdown",
                                options=[{"label": city, "value": city} for city in sales_df["City"].unique()],
                                value=['Yangon','Mandalay'],                          
                                multi=True
                                ),
                    html.H3("Month:",style={'marginTop': 9,}),
                    dcc.Dropdown(id="month-dropdown",
                                 options=[{"label": month, "value": month} for month in sales_df["month"].unique()],
                                 value=['January','February'],
                                 multi=True
                                ),
                    html.H3("Product Line:",style={'marginTop': 9,}),              
                    dcc.Dropdown(id="product-line-dropdown",
                                 options=[{"label": product, "value": product} for product in sales_df["Product line"].unique()],
                                 value=["Health and beauty","Electronic accessories"],
                                 multi=True
                                ),
                                 ], style=for_drop_div)  




for_pie_div = {'backgroundColor':'#FFFFFF',
               'justifyContent':'space-evenly',
               'width':'90%',
               'display':'flex',
                'flexDirection':'row',
              }


pie_div = html.Div(children=[dcc.Graph(id="gender-pie-chart",figure={"layout": {"height": 400, "width": 400}}),
                    dcc.Graph(id="customer-type-pie-chart",figure={"layout": {"height": 400, "width": 400}}),
                    dcc.Graph(id="payment-method-pie-chart",figure={"layout": {"height": 400, "width": 400}})
                   ], style=for_pie_div)  


for_box_div = {'backgroundColor':'#FFFFFF',
               'width':'55%',
               'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
               'margin':'0px auto',}


box_div = html.Div(children=[knopki_div,
                             dcc.Graph(id='box-output')],
                   style=for_box_div)



for_bar1_knopki = {'display':'flex',
                   'flexDirection':'row',
                   'justifyContent':'space-evenly',
                   'width':'98%',
                   'margin':'15px auto 0',}

bar1_box_div = html.Div(children=[bar1_div, box_div], 
                        style=for_bar1_knopki)



for_bar2_div = {'backgroundColor':'#FFFFFF',
                'width':'55%',
                'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
                'margin':'0px auto',}

bar2_div = dcc.Graph(figure=bar_2, 
                     style=for_bar2_div)




for_bar3_div = {'backgroundColor':'#FFFFFF',
                'width':'43%',
             'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
                'margin':'0px auto',}

bar3_div = dcc.Graph(figure=bar_3, 
                     style=for_bar3_div)



for_bar23_div ={'display':'flex',
                'flexDirection':'row',
                'justifyContent':'space-evenly',
                'width':'98%',
                'margin':'15px auto 5px',}

bar23_div = html.Div(children=[bar2_div, bar3_div], 
                     style=for_bar23_div)


for_pd_div = {'backgroundColor':'#FFFFFF',
                'borderWidth':'10px',
                  'borderColor': '#68228B',
                  'borderStyle': 'solid',
                'width':'98%',
              'margin':'15px auto 5px',
             }


Pie_drop_div=html.Div(children=[dropdown_div,pie_div],
                     style=for_pd_div)


main_div_style = {'backgroundColor':'#FFE1FF', 
                  'padding':'0', 
                  'width':'100%', 
                  'height':'100%',
                  'display':'flex',
                  'position':'fixed',
                  'flexDirection':'column',
                  'top':'0',
                  'left':'0',
                  'bottom':'0','overflow':'auto',}


app.layout = html.Div(id = "main_div", children=[sales_text, bar1_box_div,text_div, bar23_div,Pie_drop_div], 
                      style = main_div_style)


# text dropdown
@app.callback(Output('text-block-output', 'children'),
              [Input('radio-text','value')])
def update_text(value):
    return html_city_dict[value]

@app.callback(Output('dropdown-bar1-output', 'figure'),
              [Input('dropdown1-bar','value')])
def update_text(value):
    bar_drop = px.bar(data_frame=sales_df, 
                      x='City', 
                      y='ones', 
                      color=value,
                      barmode='group',
                      labels={'ones':'Number of orders'},
                      title='Number of orders (%s)' % value,
                      template='simple_white')
    bar_drop.update_traces(opacity=0.6, marker_line_width=0)
    return bar_drop

@app.callback(Output('box-output', 'figure'),
              [Input('box-drop','value'),
               Input('box-radio','value')])
def update_text(box_drop_value, box_radio_value):
    title_dict = {'cogs': 'Cost',
                  'Quantity':'Quantity',
                  'Rating':'Rating'}
    box_knop = px.box(data_frame=sales_df, 
               x='City', y=box_radio_value, color = box_drop_value,
               labels={'cogs':'Cost of goods sold, $',
                       'Quantity':'Quantity of goods sold',
                       'Rating':'Rating of goods sold'},
               title='%s of goods sold' % title_dict[box_radio_value],
               template='simple_white')
    box_knop.update_traces(opacity=0.6)
    return box_knop


# Define the callback functions for the dropdowns and pie charts
@app.callback(
    [
        dash.dependencies.Output("gender-pie-chart", "figure"),
        dash.dependencies.Output("customer-type-pie-chart", "figure"),
        dash.dependencies.Output("payment-method-pie-chart", "figure")
    ],
    [
        dash.dependencies.Input("city-dropdown", "value"),
        dash.dependencies.Input("month-dropdown", "value"),
        dash.dependencies.Input("product-line-dropdown", "value")
    ]
)
def update_pie_charts(cities, months, product_lines):
    filtered_sales_df = sales_df[
        (sales_df["City"].isin(cities)) &
        (sales_df["month"].isin(months)) &
        (sales_df["Product line"].isin(product_lines))
    ]
    
    gender_counts = filtered_sales_df["Gender"].value_counts()
    gender_fig = px.pie(
        names=gender_counts.index, 
        values=gender_counts.values, 
        title="Gender Distribution",
        template='simple_white',
        opacity=0.7
   
    )
   
    
    customer_type_counts = filtered_sales_df["Customer type"].value_counts()
    customer_type_fig = px.pie(
        names=customer_type_counts.index, 
        values=customer_type_counts.values, 
        title="Customer Type Distribution",
        template='simple_white',
        opacity=0.7
    
    )
    

    payment_method_counts = filtered_sales_df["Payment"].value_counts()
    payment_method_fig = px.pie(
        names=payment_method_counts.index, 
        values=payment_method_counts.values, 
        title="Payment Method Distribution",
        template='simple_white',
        opacity=0.7

    )
    

    return [gender_fig, customer_type_fig, payment_method_fig]


if __name__ == "__main__":
    app.run_server(port = 8060)

Dash is running on http://127.0.0.1:8060/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8060/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2023 10:36:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:57] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2023 10:36:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2023 10:36:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2023 10:36:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2023 10:36:58] "POST /_dash-update-component HTTP/1.1" 200 -
